# An easy *mitoSpotter* running pipeline for markers <img src='./webui/static/logo.png' align="right" alt="" width="120" />

All the data, scripts, and output from scripts are already in-place in our well-structured directory. Detailed file connection illustration is as follows.

![All scripts documentation](./webui/static/All_scripts_documentation.jpg)

All the preprocessed dataset (e.g., output from 01-03 scripts have been deposited in out_dir directory, but if you want to run this part, you can still do it!).

## At a glimpse: Overall script structure

/mitoSpotter/scripts  
├── 01_from_gtf_extract_id.py  
├── 02_fasta_split_by_id.py  
├── 03_sequence2unit_nt.py  
├── 04_train_hmm_nt.py  
└── 05_decode_path_nt.py  

## Before we start

- Make sure you have configure the environment correctly (described in Documentation), and selected the right kernel for this run.
- Download [human cds genome file](https://42basepairs.com/browse/web/ensembl/release-82/fasta/homo_sapiens/cds?file=Homo_sapiens.GRCh38.cds.all.fa.gz&preview=) and [GTF file](https://ftp.ensembl.org/pub/release-115/gtf/homo_sapiens/Homo_sapiens.GRCh38.115.chr.gtf.gz) into '/mitoSpotter/data' folder (this step might take a while).
- Import the package below:

In [ ]:
import sys
import runpy
from pathlib import Path
import itertools
import pandas as pd
from pathlib import Path
from IPython.display import display, Image

## STEP 1: EXTRACT NUCLEAR AND MITOCHONDRIA GENE ID

In [ ]:
sys.argv = [
    "scripts/01_from_gtf_extract_id.py",
    "--gtf", "data/Homo_sapiens.GRCh38.115.chr.gtf",
    "--outdir", "out_dir/01_ids",
    "--prefix", "human_protein_coding_marker_testing_",
    "--protein_coding_only",
]

runpy.run_path("scripts/01_from_gtf_extract_id.py", run_name="__main__")

# This equals to running 

# python scripts/01_from_gtf_extract_id.py \
#   --gtf data/Homo_sapiens.GRCh38.115.chr.gtf \
#   --outdir out_dir/01_ids \
#   --protein_coding_only \
#   --prefix human_protein_coding_

# at the terminal.

The output should look like:

![01 output](./webui/static/01_output.png)

## 02 SPLIT FASTA BY ID

In this step, we want to get the sequence with these ids.

In [ ]:
sys.argv = [
    "scripts/02_fasta_split_by_id.py",
    "--fasta", "data/Homo_sapiens.GRCh38.cds.all.fa",
    "--mito_ids", "out_dir/01_ids/human_protein_coding_marker_testing_ids_mito.txt",
    "--nuc_ids", "out_dir/01_ids/human_protein_coding_marker_testing_ids_nuclear.txt",
    "--outdir", "out_dir/02_split_fasta",
    "--prefix", "human_marker_testing_",
]

runpy.run_path("scripts/02_fasta_split_by_id.py", run_name="__main__")

The output should look like:

![02 output](./webui/static/02_output.png)

## 03 TOKENIZATION AND SPLIT INTO TRAINING AND HOLDOUT DATASETS

In [ ]:
# Define the parameter combinations
locations = ["nuclear", "mito"]
kinds = ["cds"]
modes = ["3nt", "2nt", "1nt"]

# Iterate through all combinations
for loc, kind, mode in itertools.product(locations, kinds, modes):
    sys.argv = [
        "scripts/03_sequence2unit_nt.py",
        "--fasta", f"out_dir/02_split_fasta/human_{loc}_{kind}.fa",
        "--mode", mode,
        "--train_tsv", f"out_dir/03_unit/train/human_marker_testing_{loc}_{mode}_train.tsv",
        "--holdout_tsv", f"out_dir/03_unit/holdout/human_marker_testing_{loc}_{mode}_holdout.tsv",
        "--train_frac", "0.7",
    ]
    
    print(f"Running: {loc} {kind} {mode}")
    runpy.run_path("scripts/03_sequence2unit_nt.py", run_name="__main__")

The output should look like:

![03 output](./webui/static/03_output.png)

## 04 HMM TRAINING

Yeahh! Finally comes into training phase! Let me take 3-nt level for example.

![04 param](./webui/static/04_param.png)

### EM run

In [ ]:
sys.argv = [
    "scripts/04_train_hmm_nt.py",
    "--nuclear_nt_tsv", "out_dir/03_unit/train/human_marker_testing_nuclear_3nt_train.tsv",
    "--mito_nt_tsv", "out_dir/03_unit/train/human_marker_testing_mito_3nt_train.tsv",
    "--ngram", "3",
    "--train_method", "em",
    "--learn", "et", # use et to learn both emission and transition, essential!
    "--n_em_iter", "20",
    "--out_model_json", "out_dir/04_model/human_marker_testing_3nt_model_em.json",
    "--out_vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_em.json",
    "--out_states_json", "out_dir/04_model/human_marker_testing_3nt_states_em.json",
    "--sample", "0.0001", # downsample for faster training, expected to take less than 1 minute
    "--track_memory",
    "--n_workers", "2", # if you use mac or linux, add this, otherwise remove this line
]

runpy.run_path("scripts/04_train_hmm_nt.py", run_name="__main__")

### Viterbi

In [ ]:
sys.argv = [
    "scripts/04_train_hmm_nt.py",
    "--nuclear_nt_tsv", "out_dir/03_unit/train/human_marker_testing_nuclear_3nt_train.tsv",
    "--mito_nt_tsv", "out_dir/03_unit/train/human_marker_testing_mito_3nt_train.tsv",
    "--ngram", "3",
    "--train_method", "viterbi",
    "--n_viterbi_iter", "20",
    "--out_model_json", "out_dir/04_model/human_marker_testing_3nt_model_viterbi.json",
    "--out_vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_viterbi.json",
    "--out_states_json", "out_dir/04_model/human_marker_testing_3nt_states_viterbi.json",
    "--sample", "0.0001", # downsample for faster training, expected to take less than 1 minute
    "--track_memory",
    "--n_workers", "2", # if you use mac or linux, add this, otherwise remove this line
]

runpy.run_path("scripts/04_train_hmm_nt.py", run_name="__main__")

### Hybrid

In [ ]:
# Let's make em 0.5 and viterbi 0.5 (1:1)

sys.argv = [
    "scripts/04_train_hmm_nt.py",
    "--nuclear_nt_tsv", "out_dir/03_unit/train/human_marker_testing_nuclear_3nt_train.tsv",
    "--mito_nt_tsv", "out_dir/03_unit/train/human_marker_testing_mito_3nt_train.tsv",
    "--ngram", "3",
    "--train_method", "hybrid",
    "--n_em_iter", "10",
    "--n_viterbi_iter", "10",
    "--learn", "et", # use et to learn both emission and transition, essential!
    "--out_model_json", "out_dir/04_model/human_marker_testing_3nt_model_hybird.json",
    "--out_vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_hybird.json",
    "--out_states_json", "out_dir/04_model/human_marker_testing_3nt_states_hybird.json",
    "--sample", "0.0001", # downsample for faster training, expected to take less than 1 minute
    "--track_memory",
    "--n_workers", "2", # if you use mac or linux, add this, otherwise remove this line
]

runpy.run_path("scripts/04_train_hmm_nt.py", run_name="__main__")

## 05 SEQUENCE DECODING

![05 param](./webui/static/05_param.png)

### FASTA

In [ ]:
sys.argv = [
  "05_decode_path_nt.py",

  # Load the model!
  "--model_json", "out_dir/04_model/human_marker_testing_3nt_model_em.json",
  "--vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_em.json",
  "--states_json", "out_dir/04_model/human_marker_testing_3nt_states_em.json",

  # correspondingly, use the "3-nt" mode
  "--ngram", "3",

  # An example file of fasta sequences (from a becteria which has a relatively small phylogenetic distance to mitochondria genes)
  "--fasta", "./webui/static/Rickettsia_prowazekii_str_Madrid_E.fa",

  # don't decode too short sequences (<10)
  "--min_len", "10",

  # Write output
  "--out_tsv", "out_dir/05_res/human_marker_testing_fasta_res.tsv",

  "--plotting", # plot the results! the plots will be saved in the same directory where you run the script
  "--track_memory",
]

runpy.run_path("scripts/05_decode_path_nt.py", run_name="__main__")

### STDIN

### ARGUMENT SEQUENCE PASSING

In [ ]:
sys.argv = [
  "05_decode_path_nt.py",

  # Load the model!
  "--model_json", "out_dir/04_model/human_marker_testing_3nt_model_em.json",
  "--vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_em.json",
  "--states_json", "out_dir/04_model/human_marker_testing_3nt_states_em.json",

  # correspondingly, use the "3-nt" mode
  "--ngram", "3",

  # don't decode too short sequences (<10)
  "--min_len", "10",

  # The first sequence
  "--seq", "ATACCCATGGCCAACCTCCTACTCCTCATTGTACCCATTCTAATCGCAATGGCATTCCTAATGCTTACCGAACGAAAAATTCTAGGCTATATACAACTACGCAAAGGCCCCAACGTTGTAGGCCCCTACGGGCTACTACAACCCTTCGCTGACGCCATAAAACTCTTCACCAAAGAGCCCCTAAAACCCGCCACATCTACCATCACCCTCTACATCACCGCCCCGACCTTAGCTCTCACCATCGCTCTTCTACTATGAACCCCCCTCCCCATACCCAACCCCCTGGTCAACCTCAACCTAGGCCTCCTATTTATTCTAGCCACCTCTAGCCTAGCCGTTTACTCAATCCTCTGATCAGGGTGAGCATCAAACTCAAACTACGCCCTGATCGGCGCACTGCGAGCAGTAGCCCAAACAATCTCATATGAAGTCA",
  "--seq_id", "marker_test_1",

  # The second sequence
  "--seq",
  "GTCACCCTAGCCATCATTCTACTATCAACATTACTAATAAGTGGCTCCTTTAACCTCTCCACCCTTATCACAACACAAGAACACCTCTGATTACTCCTGCCATCATGACCCTTGGCCATAATATGATTTATCTCCACACTAGCAGAGACCAACCGAACCCCCTTCGACCTTGCCGAAGGGGAGTCCGAACTAGTCTCAGGCTTCAACATCGAATACGCCGCAGGCCCCTTCGCCCTATTCTTCATAGCCGAATACACAAACATTATTATAATAAACACCCTCACCACTACAATCTTCCTAGGAACAACATATGACGCACTCTCCCCTGAACTCTACACAACATATTTTGTCACCAAGACCCTACTTCTAACCTCCCTGTTCTTATGAATTCGAACAGCATACCCCCGATTCCGCTACGACCAACTCATACACCTCCTATGAAAAAACTTCCTACCACTCACCCTAGCATTACTTATATGATATGTCTCCATACCCATTACAATCTCCAGCATTCCCCCTCAAACC",
  "--seq_id", "marker_test_2",

  # Write output
  "--out_tsv", "out_dir/05_res/human_marker_testing_arg_res.tsv",

  "--plotting", # plot the results! the plots will be saved in the same directory where you run the script
  "--track_memory",
]

runpy.run_path("scripts/05_decode_path_nt.py", run_name="__main__")

A glimpse of figures:

How many seuquences in a batch are classified as nuclear and how many are mitochondria:

![classification_counts](./webui/static/Plot_example/classification_counts.png)

GC content in each gene sequence:

![gc_content_stacked_bar](./webui/static/Plot_example/gc_content_stacked_bar.png)

Log likelihood distribution of each sequence:

![loglikelihood_distribution](./webui/static/Plot_example/loglikelihood_distribution.png)

Hidden staten proportion in each gene sequence:

![state_proportions_stacked_bar](./webui/static/Plot_example/state_proportions_stacked_bar.png)